# 1. Experimenting with a node endpoint
In this first example, we will explore how to access data from an Ethereum node.

Please create a free account with [Moralis.io](https://admin.moralis.io/register), and get your own endpoint URL. You may be able to use this notebook endpoint, but you may run into rate limits if other participants are using it at the same time. Or at a later point in time this endpoint may not work at all anymore.

In [1]:
endpoint = "http://localhost:8545"

## 1.1 Raw requests
At first, we'll be making a raw request and get a JSON response about a single block and its transactions.

In [2]:
import json
import requests

In [3]:
def create_request(method, params):
    payload = json.dumps({"method": method, "params": params, "id": 1, "jsonrpc": "2.0"})
    headers = {'content-type': "application/json"}
    try:
        response = requests.request("POST", endpoint, data=payload, headers=headers)
        return(json.loads(response.text))
    except requests.exceptions.RequestException as e:
        print(e)
    except:
        print('No response from endpoint!')

In [4]:
blockNumber = 50010
response = create_request(method="eth_getBlockByNumber", params=[hex(blockNumber),True])

## 1.2 A block JSON
Lets look at the raw JSON output of this request.
It consists of block metadata, and a list of transactions that are contained in the block.

In [5]:
blockWithTX = response["result"]
blockWithTX

{'author': '0xeb1325c8d9d3ea8d74ac11f4b00f1b2367686319',
 'difficulty': '0x15c461e952d',
 'extraData': '0x476574682f76312e302e312f77696e646f77732f676f312e342e32',
 'gasLimit': '0x4ab70',
 'gasUsed': '0xa8d8',
 'hash': '0x9b39a465d46cfcb352ea76e95037826dae53cb0d5e69098d1f55056e83eaadc4',
 'logsBloom': '0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 'miner': '0xeb1325c8d9d3ea8d74ac11f4b00f1b2367686319',
 'mixHash': '0x8f4e5db4697e918b76c487da5245e01c5ce917c068eda688c985caba9e64cdf9',
 'nonce': '0x9e2cb6340a25c0c1',
 'number':

In [6]:
len(blockWithTX["transactions"]) # How many transactions are contained in this block?

2

## 1.3 A transaction JSON

In [7]:
blockWithTX["transactions"][0]

{'blockHash': '0x9b39a465d46cfcb352ea76e95037826dae53cb0d5e69098d1f55056e83eaadc4',
 'blockNumber': '0xc35a',
 'chainId': None,
 'condition': None,
 'creates': None,
 'from': '0x86f05d19063e9369c6004eb3f123943a7cff4eab',
 'gas': '0xc350',
 'gasPrice': '0x110c8f7d8de',
 'hash': '0x79cc56eba86030510093c7f5396664f4d525b85c2efee13767fe96faf10db047',
 'input': '0x454e5057364e524638',
 'nonce': '0x0',
 'publicKey': '0x2f58cf242447014059c7bcb467645c1df89f4cf187a263ae2cbf34b3a7b1e4ee8eb464bbd6171800265ce43b2252b2f4814f9bfb7445ceef675c1cb878ddf5e2',
 'r': '0x7cbf8f47ab69d27e095737d0841e0f0697f313acd20853b89813d319ccb9a428',
 'raw': '0xf87780860110c8f7d8de82c350942910543af39aba0cd09dbb2d50200b3e800a63d28966ffcbfd5e5a30000089454e5057364e5246381ca07cbf8f47ab69d27e095737d0841e0f0697f313acd20853b89813d319ccb9a428a05db629472c555572a0260734afaa4cefa183d629d5d83fcec0ae0291a5e08062',
 's': '0x5db629472c555572a0260734afaa4cefa183d629d5d83fcec0ae0291a5e08062',
 'standardV': '0x1',
 'to': '0x2910543af39aba

In [8]:
#  r and s are outputs of an ECDSA signature, and v is the recovery id.
# https://bitcoin.stackexchange.com/questions/38351/ecdsa-v-r-s-what-is-v

## 1.4 Using web3py
At this point it's becoming clear that these json results contain mostly hexadecimal values, which is unwieldy.

Let's use web3py to have these values automatically converted to integers where applicable. This has the additional benefit that we can write less code.

In [9]:
from web3 import Web3

In [10]:
w3 = Web3(Web3.HTTPProvider(endpoint))
w3.isConnected()

True

In [11]:
blockWithTX = w3.eth.get_block(hex(blockNumber), full_transactions=True)
blockWithTX

AttributeDict({'author': '0xeb1325c8d9d3ea8d74ac11f4b00f1b2367686319',
 'difficulty': 1495825028397,
 'extraData': HexBytes('0x476574682f76312e302e312f77696e646f77732f676f312e342e32'),
 'gasLimit': 306032,
 'gasUsed': 43224,
 'hash': HexBytes('0x9b39a465d46cfcb352ea76e95037826dae53cb0d5e69098d1f55056e83eaadc4'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0xeB1325C8D9d3ea8D74aC11F4B00F1B2367686319',
 'mixHash': HexBytes('0x8f4e5db4697e918b76c487da5245e01c5ce917c068eda688c985caba9e64cdf9

In [12]:
blockWithTX.transactions[0]

AttributeDict({'blockHash': HexBytes('0x9b39a465d46cfcb352ea76e95037826dae53cb0d5e69098d1f55056e83eaadc4'),
 'blockNumber': 50010,
 'chainId': None,
 'condition': None,
 'creates': None,
 'from': '0x86F05d19063e9369c6004eB3F123943A7cfF4eaB',
 'gas': 50000,
 'gasPrice': 1171602790622,
 'hash': HexBytes('0x79cc56eba86030510093c7f5396664f4d525b85c2efee13767fe96faf10db047'),
 'input': '0x454e5057364e524638',
 'nonce': 0,
 'publicKey': HexBytes('0x2f58cf242447014059c7bcb467645c1df89f4cf187a263ae2cbf34b3a7b1e4ee8eb464bbd6171800265ce43b2252b2f4814f9bfb7445ceef675c1cb878ddf5e2'),
 'r': HexBytes('0x7cbf8f47ab69d27e095737d0841e0f0697f313acd20853b89813d319ccb9a428'),
 'raw': HexBytes('0xf87780860110c8f7d8de82c350942910543af39aba0cd09dbb2d50200b3e800a63d28966ffcbfd5e5a30000089454e5057364e5246381ca07cbf8f47ab69d27e095737d0841e0f0697f313acd20853b89813d319ccb9a428a05db629472c555572a0260734afaa4cefa183d629d5d83fcec0ae0291a5e08062'),
 's': HexBytes('0x5db629472c555572a0260734afaa4cefa183d629d5d83fcec0a

Notice how the **from** and **to** addresses are now partially uppercase?
These are the checksummed versions of the lowercase addreses. This case-sensivity is used for checksum validation. The address is compared against the raw binary keccak-256 hash of the address bytes, and where there are letters in the same corresponding place as a "1" bit, the letter is capitalized. Just make sure you either always use checksum addreses or lowercase addresses. Otherwise your analysis might perceive the same address as two different ones!

## 1.5 Let's turn the transaction JSON into a dataframe

In [13]:
import pandas as pd
transactionDF = pd.DataFrame(blockWithTX.transactions)[["from", "to", "value"]]
transactionDF

,from,to,value
0,0x86F05d19063e9369c6004eB3F123943A7cfF4eaB,0x2910543Af39abA0Cd09dBb2D50200b3E800A63D2,1900000000000000000000
1,0x3F98e477a361F777DA14611a7e419A75Fd238b6b,0x2910543Af39abA0Cd09dBb2D50200b3E800A63D2,500000000000000000000


You can already see that the value column seems to contain very large numbers. This is because they are in the unit **Wei**. One Ether corresponds to 10^18 Wei. So we can convert the column accordingly.

In [14]:
transactionDF["value"] = transactionDF["value"].apply(lambda x: x/10**18)

In [15]:
transactionDF

,from,to,value
0,0x86F05d19063e9369c6004eB3F123943A7cfF4eaB,0x2910543Af39abA0Cd09dBb2D50200b3E800A63D2,1900.0
1,0x3F98e477a361F777DA14611a7e419A75Fd238b6b,0x2910543Af39abA0Cd09dBb2D50200b3E800A63D2,500.0
